# Pandas и большие файлы

In [78]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

# Объединение датафреймов

### Данные со слайдов

In [82]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [3]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [87]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped = visits_grouped.reset_index()
visits_grouped

,user_id,visits
0,11,2
1,22,1
2,55,1
3,77,1


In [11]:
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [111]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', aggfunc='size', fill_value=0)
purchases_pivot

# Если категорий, которые стали столбцами, не очень много (не сотни и не тысячи), то такой способ создания сводных таблиц
# предпочтителен. Мы сохраняем максимум информации из тех данных, которые есть на входе. Единственное замечание -
# у нас таблица довольно маленькая, в плане количества столбцов. И при построении таких сводных таблиц могут быть
# проблемы с наименованием столбцов. Иногда при построении сводных таблиц придется добавлять одни и те же столбцы
# (если у нас их мало) и в строчки, и в значения. И в некоторых случаях придется заменить стандартную функцию 'count'
# на 'size', чтобы сводная таблица была построена. Поэтому если у вас есть сводная таблица с малым числом столбцов,
# то иногда это, к сожалению, вызывает проблемы. С этим нужно дополнительно в каждом случае разбираться.

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [118]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [119]:
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [120]:
visits_grouped

,user_id,visits
0,11,2
1,22,1
2,55,1
3,77,1


In [122]:
visits_grouped.join(purchases_pivot)

,user_id,visits,Авто,Дача,Спорт
0,11,2,NaN,NaN,NaN
1,22,1,NaN,NaN,NaN
2,55,1,NaN,NaN,NaN
3,77,1,NaN,NaN,NaN


In [124]:
visits_grouped.merge(purchases_pivot, on = 'user_id')

,user_id,visits,Авто,Дача,Спорт
0,11,2,0,0,2
1,22,1,1,0,0
2,55,1,0,1,0


Итак, мы взяли исходные данные, заметили, что в них есть дубликаты, и удалили их двумя способами.

    Первый - простая группировка: просто группируем по столбцу, по которому будем в будущем объединять. И получаем простую короткую таблицу, где у нас есть все визиты, но нет одного столбца по источникам трафика.

    Второй вариант, более продвинутый: можно один из столбцов сохранить в данных, используя сводную таблицу.
    
 Результат группировки - один и тот же. Но во втором случае сохраняется дополнительная информация, которая в будущем может пригодиться.

## Типы объединений в Pandas

Метод .join используется тогда, когда нужно склеить датафреймы по индексу (т.е. тот столбец, по которому склеиваем, становится индексом). В принципе, его можно легко вернуть в столбцы с помощью метода .reset_index(), но если этого не сделать, то объединять эти датафреймы можно по индексу user_id.

Этот метод довольно удобен. Индексы чаще всего уникальные и есть даже набор оптимизаций, когда можно ускорить какие-то очень хитрые объединения, переведя эти столбцы в индекс. Т.к. операция по объединению таблиц - довольно прожорлива по ресурсам (эта проблема есть и в Pandas, и в SQL). А если перевести столбцы в индекс, то есть параметры, которые позволяют этот процесс существенно ускорить.

И в этом случае используется метод .join, прямо точно так же, как в sql-ных базах данных.

Но есть и другой способ. Это метод .merge, когда в качестве столбцов, по которым будет происходить объединение, указывается не индекс, а название столбцов. Т.е. если мы переведем столбец 'user_Id' из индекса в столбец с помощью .reset_index(), то теперь для объединения таблиц мы должны использовать столбец 'user_Id', а не индекс. И в данном случае используется метод .merge().

Это особенность Pandas и иногда люди, переходящие на Pandas с SQL, не могут к этому привыкнуть, постоянно пишут .join и не очень понятно, почему Pandas выдает ошибку.

In [21]:
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [22]:
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [27]:
visits_grouped.join(purchases_pivot)

# Для пользователей 11, 22, 55 все выглядит красиво.
# Но вот пользователь 77 из таблицы визитов отображается с 1 визитом и пустыми категориями.
# И есть пользователь 99, который был в категориях покупок. Его в объединенной таблице нет вообще.
# И как-то не очень понятно, что же за результат мы получили.

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [23]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой.
Данный метод не рекомендуется к использованию без особой необходимости: операции объединения и так сложны для мозга, и все привыкли к левому объединению. Правое объединение заставляет мозг напрягаться избыточно, чтобы понять, что происходит.

In [24]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


А что делать, если нам надо все данные из обеих таблиц сохранить, мы не хотим ничего терять? Нам для этого не подойдет ни left join, ни right join.
Для этого есть еще 2 типа объединения таблиц, тоже базовых: iner и outer join. Они тоже есть и в Pandas, и в большинстве баз данных. Для них нет разницы, какая таблица слева, а какая справа.

### INNER join
Оставляет строчки, которые есть в обеих таблицах.

In [28]:
visits_grouped.join(purchases_pivot, how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки.

In [29]:
visits_grouped.join(purchases_pivot, how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [ ]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

In [ ]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

In [ ]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

# Конкатенация таблиц

В SQL есть аналог этому - оператор UNION ALL, когда таблицы или результаты запросов просто подставляются один к другому.

In [127]:
# Эту операцию мы уже видели, например, при использовании строк:
'abc' + 'def'

'abcdef'

In [128]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [32]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [129]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [130]:
pd.concat([a, b])

,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


In [133]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

# при таком объединении может получиться, что каких-то столбцов несколько (например 'date' здесь)
# но Pandas считает это нормальной ситуацией, и никаких ошибок не возникает.
# pd.concat([a, b], axis=1)['date']
# Но в практических задачах такое построчное склеивание нужно довольно редко.
# Хотя в исключительных случаях это сильно упрощает жизнь.

,date,value_a,date,value_b
0,2020-01-01,1,2020-01-01,3
1,2020-01-02,2,2020-01-02,4
2,2020-01-03,3,2020-01-03,5


### Дубликаты при объединении таблиц

Давайте посмотрим, когда вы мало знакомы с данными, и нам надо представить, в каких вообще ситуациях дубликаты появляются.

In [134]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [135]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

# Здеь movieId 31 встречается дважды. И это похоже на ошибку.

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [136]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

# В этом случае Pandas возьмет единственную строчку датафрейма ratings, возьмет значение 31
# и пойдет его искать в правом датафрейме movies, и, естественно, найдет его дважды, в строке 1 и 3.
# И вроде ничего страшного: дважды нашел, дважды подставил. В чем проблема?
# Она в том, что статистические данные левого датафрейма ratings (про оценку и timestamp), задвоятся.
# И правильно посчитать, например, количество оценок, уже не получится. И средний рейтинг при таком задвоении тоже
# будет считаться неверно. # И это классическая проблема, когда задвоение не учли, но после объединения оно есть,
# и вычисления нарушаются. # И с этим нужно бороться. Как?

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [137]:
# Для разных типов задач проблему дублей нужно решать по-разному. Например, через группировки и сводные таблицы.
# В данном случае лучше всего - удалить лишние строки, чтобы они не мешали, с помощью метода .drop_duplicates
# Он берет датафрейм и указывает столбец, по которому ищутся дубликаты. И эти дубликаты удаляются.
# Чтобы понять, какой дубликат оставлять, есть параметр keep. Его варианты: 'first', 'last'.

movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [138]:
# Метод .drop_duplicates может использоваться для удаления дубликатов сразу в нескольких столбцах.
# Но это не значит, что Pandas будет заходить в каждый из столбцов и искать там дубликаты.
# Он будет искать те строки заданных столбцов, которые повторяют друг друга.
# Если указать все столбцы датафрейма, то метод будет искать строки, которые идеально совпадают.
# Если параметр subset не указать совсем, то как раз и будет этот случай: Pandas будет искать совпадающие строки. 

movies.drop_duplicates(subset = ['movieId', 'title'], keep = 'first', inplace = True)

In [139]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [140]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [ ]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

In [ ]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

### Оптимизация хранения данных

Поговорим про то, зачем очень часто разные таблицы хранятся в разных файлах или разных таблицах базы данных.

In [141]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [48]:
joined.head()

# Одна такая таблица - это же гораздо удобнее, чем хранить данные о фильмах и их рейтингах отдельно.
# Но так почему-то не делают. И на это есть несколько логичных причин.

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [142]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [143]:
# Длина датафрейма с рейтингами - довольно большая. При этом каждая строка короткая и состоит из целых чисел.
# А значит, она занимает мало места в оперативной памяти и на жестком диске при хранении в файлах.

len(ratings)

100004

In [144]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [52]:
# А вот длина датафрйма movies - гораздо меньше. Но эти строчки длинные.
# Т.е. каждая строка занимает довольно много оперативной памяти и места на жестком диске, когда они хранятся в файлах.

len(movies)

9125

In [53]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

In [145]:
# И получается, что в общей таблице для каждой оценки лога, которая весит мало, есть какая-то тяжелая часть,
# с расшифровками, которая многократно повторяется (поскольку оценки фильма делают разные пользователи).

joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [146]:
# Поэтому что обычно делают администраторы баз данных и разработчики? Они делают логичную вещь:
# Давайте каждой длинной str, которая многократно повторяется, придумаем короткий айдишник.
# В данном случае эту роль играет столбец 'movieId'

ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


И разделим эти таблицы на 2: одна из них будет содержать строчки лога, но вместо названий - короткие айдишники (наша таблица ratings). А вторая часть - короткая таблица с длинными названиями, но уже без дубликатов.

In [147]:
joined.to_csv('joined_ratings.csv', index=False)

Даже если на этом учебном примере посмотреть, сколько весит файл с объединенными данными, то окажется, что он весит 7 мб, а обе таблицы, которые мы объединяли, суммарно весят менее 3 мб. Получается, что после разделения таблиц мы более чем в 2 раза сэкономили места. На реальных данных разница будет еще больше. Поэтому такая схема с разделением данных (она называется "снежинка") очень сильно помогает экономить место.

И когда готовится нужный отчет, то данные склеиваются. При этом необязательно брать всю таблицу, достаточно взять нужные столбцы.

In [63]:
logs = joined[['userId', 'movieId', 'rating']]
logs.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [64]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

9066

### Какой жанр имеет самые высокие рейтинги?
(посчитаем рейтинг жанров)

In [65]:
import numpy as np

In [66]:
genres = ['Drama', 'Action', 'Thriller']

In [67]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [68]:
len(ratings)

100004

In [69]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [70]:
len(movies)

9125

In [71]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [72]:

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [73]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [74]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [75]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

Wall time: 16.6 s


In [76]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [77]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN


### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base